# MultiClass AdaBoost with ID3 as Base Learner


This notebook demonstrates the implementation of the AdaBoost algorithm using the ID3 decision tree as the base learner. The task involves multi-class classification on the Letter Recognition dataset.


## Step 1: Load and Preprocess the Data

In [136]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
columns = [
    'letter', 'x-box', 'y-box', 'width', 'high', 'onpix', 'x-bar', 
    'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 
    'x-ege', 'xegvy', 'y-ege', 'yegvx'
]
data = pd.read_csv('letter-recognition.data', header=None, names=columns)

# Display the first few rows of the dataset to verify its structure
data.head()



,letter,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [137]:
# Map letters to numeric classes for easier processing
data['letter'] = data['letter'].apply(lambda x: ord(x) - ord('A'))

# Separate features and target variable
X = data.iloc[:, 1:]
y = data['letter']

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16000, 16), (4000, 16), (16000,), (4000,))

In [138]:
X_train.head()


,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
7563,6,10,8,8,9,8,7,6,3,7,8,8,10,6,6,11
6911,5,8,5,6,3,4,8,5,7,11,10,13,1,9,3,8
3307,4,7,6,8,5,8,8,5,6,7,7,7,4,8,9,9
11099,4,9,6,7,8,7,6,3,2,7,5,7,3,7,12,2
18859,1,3,2,2,1,7,7,1,8,11,6,9,1,9,5,8


In [139]:
X_test.head()


,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
2734,3,5,6,4,4,9,7,3,5,9,4,6,3,7,4,10
2550,5,10,6,8,4,7,7,12,2,8,9,8,8,6,0,8
3250,7,10,9,8,7,9,7,4,7,10,5,6,2,8,6,10
18549,2,4,3,3,1,7,9,3,4,12,5,3,1,10,2,8
6283,1,1,1,1,0,12,3,6,4,13,4,11,0,7,0,8


In [140]:
y_train.head()


7563     22
6911      2
3307      8
11099    18
18859    25
Name: letter, dtype: int64

In [141]:
y_test.head()

2734     17
2550     12
3250      1
18549    15
6283      9
Name: letter, dtype: int64

## Step 2: Implement the ID3 Decision Tree Algorithm

In [142]:

class ID3DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def _entropy(self, y):
        unique, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-9))

    def _information_gain(self, X_column, y, threshold):
        parent_entropy = self._entropy(y)
        left_idx = X_column <= threshold
        right_idx = X_column > threshold
        if len(y[left_idx]) == 0 or len(y[right_idx]) == 0:
            return 0
        n = len(y)
        n_left, n_right = len(y[left_idx]), len(y[right_idx])
        e_left, e_right = self._entropy(y[left_idx]), self._entropy(y[right_idx])
        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right
        return parent_entropy - child_entropy

    def _best_split(self, X, y):
        best_gain = -1
        best_feature = None
        best_threshold = None
        for feature_index in range(X.shape[1]):
            X_column = X[:, feature_index]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(X_column, y, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_index
                    best_threshold = threshold
        return best_feature, best_threshold

    def _build_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        if depth == self.max_depth or n_labels == 1 or n_samples == 0:
            return np.bincount(y).argmax()
        feature_index, threshold = self._best_split(X, y)
        if feature_index is None:
            return np.bincount(y).argmax()
        left_idxs = X[:, feature_index] <= threshold
        right_idxs = X[:, feature_index] > threshold
        left_subtree = self._build_tree(X[left_idxs], y[left_idxs], depth + 1)
        right_subtree = self._build_tree(X[right_idxs], y[right_idxs], depth + 1)
        return (feature_index, threshold, left_subtree, right_subtree)

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, 0)

    def _traverse_tree(self, x, tree):
        if not isinstance(tree, tuple):
            return tree
        feature_index, threshold, left, right = tree
        if x[feature_index] <= threshold:
            return self._traverse_tree(x, left)
        return self._traverse_tree(x, right)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])


## Step 3: Implement the AdaBoost Algorithm

In [ ]:

    class MultiClassAdaBoost:
        def __init__(self, base_learner_class, n_estimators=50):
            self.base_learner_class = base_learner_class
            self.n_estimators = n_estimators
            self.models = []
            self.alphas = []
            self.classes = None

        def fit(self, X, y):
            n_samples = X.shape[0]
            self.classes = np.unique(y)
            n_classes = len(self.classes)
            weights = np.ones(n_samples) / n_samples
            for _ in range(self.n_estimators):
                model = self.base_learner_class(max_depth=5)
                model.fit(X, y)
                y_pred = model.predict(X)
                incorrect = (y_pred != y).astype(int)
                error = np.dot(weights, incorrect) / np.sum(weights)
                if error >= 0.5 or error == 0:
                    break
                alpha = 0.5 * np.log((1 - error) / error)
                weights = weights * np.exp(alpha * incorrect)
                weights /= np.sum(weights)
                self.models.append(model)
                self.alphas.append(alpha)

        def predict(self, X):
            class_votes = np.zeros((X.shape[0], len(self.classes)))
            for alpha, model in zip(self.alphas, self.models):
                predictions = model.predict(X)
                for i, pred in enumerate(predictions):
                    class_index = np.where(self.classes == pred)[0][0]
                    class_votes[i, class_index] += alpha
            return self.classes[np.argmax(class_votes, axis=1)]


## Step 4: Train and Evaluate the Model

In [144]:

# Train MultiClass AdaBoost on a subset of data for demonstration
multiclass_adaboost = MultiClassAdaBoost(base_learner_class=ID3DecisionTree, n_estimators=10)
multiclass_adaboost.fit(X_train.values[:6500], y_train.values[:6500])

# Predict on a small subset of test data
multiclass_predictions = multiclass_adaboost.predict(X_test.values)

# Display predictions
multiclass_predictions

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, multiclass_predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n")
print(classification_report(y_test, multiclass_predictions))


Accuracy: 49.98%

Classification Report:

              precision    recall  f1-score   support

           0       0.78      0.75      0.77       158
           1       0.33      0.75      0.46       153
           2       0.69      0.65      0.67       147
           3       0.57      0.61      0.59       161
           4       0.42      0.47      0.44       154
           5       0.65      0.18      0.28       155
           6       0.25      0.25      0.25       155
           7       0.76      0.25      0.38       147
           8       0.67      0.59      0.63       151
           9       0.99      0.50      0.66       149
          10       0.15      0.35      0.21       148
          11       0.75      0.66      0.70       152
          12       0.98      0.68      0.81       158
          13       0.86      0.57      0.69       157
          14       0.00      0.00      0.00       150
          15       0.47      0.83      0.60       161
          16       0.28      0.72      

c:\Users\sadma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sadma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sadma\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
